# Clasificador de datos que permita reconocer la marcha - Hold Out

## 0. Initial Setup (carga de libreriías)

In [1]:
import import_ipynb
from functions import * 
from ciervo.plots import emg_plot
import torch.nn as nn

importing Jupyter notebook from functions.ipynb
fatal: destination path 'balu3' already exists and is not an empty directory.
Processing ./balu3
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for balu3: filename=balu3-1.0-py3-none-any.whl size=43718 sha256=86f904a8293e49d3a2022da14cd67aacca9451a7aaf012ce9e2b73f79e998873
  Stored in directory: /tmp/pip-ephem-wheel-cache-12s6gj7q/wheels/1d/f1/b3/dfbb6908bffc070ba2baad77932ab66db0022561f2c4127596
Successfully built balu3
  Attempting uninstall: balu3
    Found existing installation: balu3 1.0
    Uninstalling balu3-1.0:
      Successfully uninstalled balu3-1.0


In [ ]:
!nvidia-smi

## 1. Data Loading

In [2]:
data_files = load_data()

## 2. Feature Extraction 


Preparando datos

In [3]:
step, emg, file= prepare_data(data_files)

<string>:41: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


## 3.Training and testing subsets // Classification & Evaluation

In [4]:
# Definir las fases a probar
fases_to_test = [4, 6, 8, 10, 12, 14, 16]

# Crear listas para almacenar los resultados
results_simple_nn = []
results_cnn = []
results_rnn = []

In [ ]:
for num_fases in fases_to_test:
    print(f"TESTING WITH {num_fases} PHASES...")
    
    train_data, train_labels, test_data, test_labels, train_feature_names, test_feature_names = prepare_training_and_testing(emg, file, C=[0,1,2], divide=3, num_fases=num_fases)
    train_data, test_data = clean_normalized_feature_selection(train_data, test_data)
    train_data, test_data = sfs_selection(train_data,test_data,train_labels,n_indices=5)
    print(f"Train data: Extracted {train_data.shape[0]} samples with {train_data.shape[1]} features each.")
    print(f"Test data: Extracted {test_data.shape[0]} samples with {test_data.shape[1]} features each.")
    print("-----------------------------------------------------------------------")

    # Simple NN
    print("Simple NN")
    input_size = train_data.shape[1]
    hidden_size = 100
    output_size = num_fases
    learning_rate = 0.001
    num_epochs = 100
    
    model = SimpleNN(input_size, hidden_size, output_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    train_loader, test_tensor, test_labels_tensor = prepare_data_SimpleNN(train_data, train_labels, test_data, test_labels)
    train_model_SimpleNN(model, criterion, optimizer, train_loader, num_epochs)
    accuracy_nn = evaluate_model_SimpleNN(model, test_tensor, test_labels_tensor)
    results_simple_nn.append((num_fases, accuracy_nn))

    # CNN
    print("-----------------------------------------------------------------------")
    print("CNN")
    train_loader, test_loader, test_labels_tensor, train_data_tensor = prepare_data_CNN(train_data, train_labels, test_data, test_labels)
    input_length = train_data_tensor.shape[2]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    cnn_model = GaitCNN(input_length, num_fases=num_fases).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001)
    
    train_loss, test_loss = train_model_CNN(cnn_model, device, criterion, optimizer, train_loader, test_loader, num_epochs=100)
    accuracy_cnn = evaluate_model_CNN(cnn_model, test_loader, test_labels_tensor)
    results_cnn.append((num_fases, accuracy_cnn))

    # RNN
    print("-----------------------------------------------------------------------")
    print("RNN")
    train_data_rnn, test_data_rnn = preprocess_data_RNN(train_data, test_data)
    input_size = train_data_rnn.shape[2]
    hidden_size = 64
    num_layers = 2
    batch_size = 32
    num_epochs = 150
    
    train_loader, test_loader = create_dataloaders_RNN(train_data_rnn, train_labels, test_data_rnn, test_labels, batch_size)
    rnn_model = GaitRNN(input_size, hidden_size, num_layers, num_fases).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(rnn_model.parameters(), lr=0.001)
    
    train_loss, test_loss = train_model_RNN(rnn_model, train_loader, test_loader, criterion, optimizer, num_epochs, device)
    accuracy_rnn = evaluate_model_RNN(rnn_model, test_loader, test_labels, device)
    results_rnn.append((num_fases, accuracy_rnn))
    print("-----------------------------------------------------------------------")


TESTING WITH 4 PHASES...
[28 26  8  5 14]
Train data: Extracted 5156 samples with 5 features each.
Test data: Extracted 2620 samples with 5 features each.
-----------------------------------------------------------------------
Simple NN
Epoch [10/100], Loss: 0.6961105465888977
Epoch [20/100], Loss: 0.0755750983953476
Epoch [30/100], Loss: 0.4163396954536438
Epoch [40/100], Loss: 0.6866856217384338
Epoch [50/100], Loss: 0.1482744663953781
Epoch [60/100], Loss: 0.06431372463703156
Epoch [70/100], Loss: 0.5354350209236145
Epoch [80/100], Loss: 0.03961857035756111
Epoch [90/100], Loss: 1.2552785873413086
Epoch [100/100], Loss: 0.2148297280073166
Accuracy: 0.7015267175572519%
-----------------------------------------------------------------------
CNN
Epoch [10/100], Train Loss: 0.37176113695274166, Val Loss: 0.6560580068972053
Epoch [20/100], Train Loss: 0.33670889521822517, Val Loss: 0.8003597533920916
Epoch [30/100], Train Loss: 0.31879787524173286, Val Loss: 0.7232423613711101
Epoch [40/

Epoch [10/100], Loss: 0.7411975860595703
Epoch [20/100], Loss: 0.8029157519340515
Epoch [30/100], Loss: 1.3836233615875244
Epoch [40/100], Loss: 1.5546725988388062
Epoch [50/100], Loss: 1.0263904333114624
Epoch [60/100], Loss: 1.341202735900879
Epoch [70/100], Loss: 0.9783419966697693
Epoch [80/100], Loss: 0.9092841744422913
Epoch [90/100], Loss: 0.9814821481704712
Epoch [100/100], Loss: 0.9890117645263672
Accuracy: 0.49446564885496186%
-----------------------------------------------------------------------
CNN
Epoch [10/100], Train Loss: 1.1282502523516722, Val Loss: 1.2140975819855202
Epoch [20/100], Train Loss: 1.0721264618100028, Val Loss: 1.1891033184237596
Epoch [30/100], Train Loss: 1.048373462061395, Val Loss: 1.184344907722822
Epoch [40/100], Train Loss: 1.0258272902884351, Val Loss: 1.1858930253401034
Epoch [50/100], Train Loss: 1.0099987916348518, Val Loss: 1.186248037509802
Epoch [60/100], Train Loss: 1.0002240156610684, Val Loss: 1.2234143763780594
Epoch [70/100], Train Lo

Epoch [60/100], Loss: 1.438555121421814
Epoch [70/100], Loss: 1.3262298107147217
Epoch [80/100], Loss: 1.875027060508728
Epoch [90/100], Loss: 1.0514928102493286
Epoch [100/100], Loss: 1.315033197402954
Accuracy: 0.3278625954198473%
-----------------------------------------------------------------------
CNN
Epoch [10/100], Train Loss: 1.5723645320115995, Val Loss: 1.7447690920131962
Epoch [20/100], Train Loss: 1.5217035563031505, Val Loss: 1.7480929760428947
Epoch [30/100], Train Loss: 1.4990660673823237, Val Loss: 1.7383572139391086
Epoch [40/100], Train Loss: 1.484052272621265, Val Loss: 1.7287688987041876
Epoch [50/100], Train Loss: 1.4741289864394291, Val Loss: 1.7195617091364976
Epoch [60/100], Train Loss: 1.464356103711877, Val Loss: 1.7284404994026432
Epoch [70/100], Train Loss: 1.4551661930793574, Val Loss: 1.7321648844858495
Epoch [80/100], Train Loss: 1.4515416845063533, Val Loss: 1.7120155881090862
Epoch [90/100], Train Loss: 1.4465007562775256, Val Loss: 1.7037781724115697


Epoch [40/100], Train Loss: 1.8589500351469646, Val Loss: 2.0250981566382618
Epoch [50/100], Train Loss: 1.8464743203895038, Val Loss: 2.022738781280634
Epoch [60/100], Train Loss: 1.8404945538025494, Val Loss: 2.0613836482530687
Epoch [70/100], Train Loss: 1.8363509873087092, Val Loss: 2.0167386775336613
Epoch [80/100], Train Loss: 1.8283852978270183, Val Loss: 2.0251910701030638
Epoch [90/100], Train Loss: 1.8222959448200788, Val Loss: 2.0127688987952905
Epoch [100/100], Train Loss: 1.820206405950147, Val Loss: 2.0157938599586487
Accuracy of CNN: 0.2583969465648855
-----------------------------------------------------------------------
RNN
Epoch 5/150, Train Loss: 1.997800713731337, Val Loss: 2.0933684659440344
Epoch 10/150, Train Loss: 1.8854786837747854, Val Loss: 2.012128849218531
Epoch 15/150, Train Loss: 1.8581903923389524, Val Loss: 1.9834106019357356
Epoch 20/150, Train Loss: 1.8393411590147388, Val Loss: 2.0172652729400773
Epoch 25/150, Train Loss: 1.8275355666182762, Val Los

In [ ]:
# Mostrar los resultados
print("Results for Simple NN:")
for num_fases, acc in results_simple_nn:
    print(f"Phases: {num_fases}, Accuracy: {acc}")

print("Results for CNN:")
for num_fases, acc in results_cnn:
    print(f"Phases: {num_fases}, Accuracy: {acc}")

print("Results for RNN:")
for num_fases, acc in results_rnn:
    print(f"Phases: {num_fases}, Accuracy: {acc}")